In [1]:
# import modul
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [6]:
# one hot encoding
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [7]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [8]:
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [36]:
tokenizer = Tokenizer(num_words=6000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sequence_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sequence_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sequence_latih)
padded_test = pad_sequences(sequence_test)

In [37]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim=6000, output_dim=16),
                                    tf.keras.layers.LSTM(64),
                                    
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(64, activation='relu'),
                                    tf.keras.layers.Dense(5, activation='softmax')])

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 16)          96000     
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                20736     
_________________________________________________________________
dense_21 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_22 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_23 (Dense)             (None, 5)                 325       
Total params: 133,637
Trainable params: 133,637
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(padded_latih, label_latih, validation_data=(padded_test, label_test), epochs=30)

Epoch 1/30
26/26 [==============================] - 73s 939ms/step - loss: 1.6114 - accuracy: 0.1878 - val_loss: 1.6114 - val_accuracy: 0.1443
Epoch 2/30
26/26 [==============================] - 24s 918ms/step - loss: 1.6067 - accuracy: 0.2400 - val_loss: 1.6109 - val_accuracy: 0.1393
Epoch 3/30
26/26 [==============================] - 24s 919ms/step - loss: 1.5516 - accuracy: 0.3321 - val_loss: 1.5970 - val_accuracy: 0.2139
Epoch 4/30
26/26 [==============================] - 24s 915ms/step - loss: 1.3625 - accuracy: 0.3644 - val_loss: 1.6523 - val_accuracy: 0.2040
Epoch 5/30
26/26 [==============================] - 24s 911ms/step - loss: 1.0757 - accuracy: 0.4490 - val_loss: 1.8860 - val_accuracy: 0.1791
Epoch 6/30
26/26 [==============================] - 24s 906ms/step - loss: 0.9203 - accuracy: 0.5535 - val_loss: 2.0189 - val_accuracy: 0.2687
Epoch 7/30
26/26 [==============================] - 23s 902ms/step - loss: 0.7130 - accuracy: 0.6654 - val_loss: 3.0361 - val_accuracy: 0.2040